In [4]:
import numpy as np
from pathlib import Path
from PIL import Image, ImageEnhance, ImageFilter
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

# Load the image
IMAGE_PATH = Path("downscaled_data/test_image.jpg")  # Replace with your image path

def preprocess_image(contrast_factor, threshold, diff_threshold):
    """Preprocess an image with adjustable parameters."""
    with Image.open(IMAGE_PATH) as img:
        # Step 1: Enhance contrast
        enhancer = ImageEnhance.Contrast(img)
        img = enhancer.enhance(contrast_factor)

        # Step 2: Remove noise using a median filter
        img = img.filter(ImageFilter.MedianFilter(size=3))

        # Step 3: Apply binarization with the given formula
        img_array = np.array(img)
        if img_array.ndim == 3:  # Ensure the image has multiple channels (e.g., RGB)
            r, g, b = img_array[..., 0], img_array[..., 1], img_array[..., 2]
            condition = (
                (r < threshold) & (g < threshold) & (b < threshold) &
                (np.max(np.stack([np.abs(r - g), np.abs(r - b), np.abs(g - b)], axis=0), axis=0) < diff_threshold)
            )
            binary_array = (1 - condition.astype(np.uint8)) * 255  # Convert to binary (0 or 255)
        else:
            raise ValueError("Input image must have 3 channels (RGB).")

        binary_img = Image.fromarray(binary_array, mode="L")  # Convert back to grayscale image
        return binary_img

def update_image(contrast_factor, threshold, diff_threshold):
    """Update the displayed image based on widget values."""
    binary_img = preprocess_image(contrast_factor, threshold, diff_threshold)
    plt.figure(figsize=(7, 15))
    plt.axis("off")
    plt.imshow(binary_img, cmap="gray")
    plt.show()

# Create widgets
contrast_slider = widgets.FloatSlider(value=1.5, min=0.5, max=3.0, step=0.1, description="Contrast:")
threshold_slider = widgets.IntSlider(value=150, min=0, max=255, step=1, description="Threshold:")
diff_threshold_slider = widgets.IntSlider(value=100, min=0, max=255, step=1, description="Diff Threshold:")

# Display widgets and image
ui = widgets.VBox([contrast_slider, threshold_slider, diff_threshold_slider])
out = widgets.interactive_output(update_image, {
    'contrast_factor': contrast_slider,
    'threshold': threshold_slider,
    'diff_threshold': diff_threshold_slider
})

display(ui, out)

Output()